In [1]:
from decimal import ROUND_HALF_UP, Decimal

import numpy as np
import pandas as pd

In [3]:
df_price = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv')
df_financial = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/train_files/financials.csv')
df_stocklist = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/stock_list.csv')

/var/folders/1k/wj0bvnz165sfyvwsqlfxwfmw6bk41g/T/ipykernel_83884/1116028146.py:2: DtypeWarning: Columns (14,15,16,17,18,19,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df_financial = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/train_files/financials.csv')


In [4]:
def adjust_price(price):
    """
    Args:
        price (pd.DataFrame)  : pd.DataFrame include stock_price
    Returns:
        price DataFrame (pd.DataFrame): stock_price with generated AdjustedClose
    """
    # transform Date column into datetime
    price.loc[: ,"Date"] = pd.to_datetime(price.loc[: ,"Date"], format="%Y-%m-%d")

    def generate_adjusted_close(df):
        
        # sort data to generate CumulativeAdjustmentFactor
        df = df.sort_values("Date", ascending=False)
        # generate CumulativeAdjustmentFactor
        df.loc[:, "CumulativeAdjustmentFactor"] = df["AdjustmentFactor"].cumprod()
        # generate AdjustedClose
        df.loc[:, "AdjustedClose"] = (
            df["CumulativeAdjustmentFactor"] * df["Close"]
        ).map(lambda x: float(
            Decimal(str(x)).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP)
        ))
        # reverse order
        df = df.sort_values("Date")
        # to fill AdjustedClose, replace 0 into np.nan
        df.loc[df["AdjustedClose"] == 0, "AdjustedClose"] = np.nan
        # forward fill AdjustedClose
        df.loc[:, "AdjustedClose"] = df.loc[:, "AdjustedClose"].ffill()
        return df

    # generate AdjustedClose
    price = price.sort_values(["SecuritiesCode", "Date"])
    price = price.groupby("SecuritiesCode").apply(generate_adjusted_close).reset_index(drop=True)

    # price.set_index("Date", inplace=True)
    return price

In [5]:
price_cols = ['Close']

df_price[price_cols] = df_price[price_cols].replace(0.0, np.nan)

# add high/low
df_price["diff_high_low"] = (
    np.log1p(df_price["High"]) / np.log1p(df_price["Low"])
    ).astype("float16")
    
log_price = df_price[price_cols].apply(np.log1p)
diff_log_price = log_price.diff()
diff_log_price['SecuritiesCode'] = df_price.SecuritiesCode

# add new features
fea_price = [df_price]

group_code = df_price.groupby("SecuritiesCode")
price_gb_code = df_price.groupby("SecuritiesCode")[price_cols]
diff_log_price_gb_code = diff_log_price.groupby("SecuritiesCode")[price_cols]

for span in [20, 40, 60, 120]:
    pct_change = (
        price_gb_code.pct_change(span)
            .add_prefix(f"feat_pct{span}_")
            .astype('float16')
    )
    volatility = (
        diff_log_price_gb_code.rolling(span, min_periods=1)
            .std()
            .reset_index("SecuritiesCode", drop=True)
    )
    volatility = (
        (log_price / volatility).add_prefix(f"feat_vl{span}_")
            .astype('float16')
    )
    ma = (
        price_gb_code.rolling(span, min_periods=1)
            .mean()
            .reset_index("SecuritiesCode", drop=True)
    )
    ma = (df_price[price_cols] / ma).add_prefix(f"feat_rmr{span}_").astype('float16')
    diff_roll = (
        group_code.diff_high_low
            .rolling(span, min_periods=1)
            .mean().
            rename(f"feat_wd{span}")
            .reset_index("SecuritiesCode", drop=True)
            .astype('float16')
    )
    fea_price += [pct_change, volatility, ma, diff_roll]

df_price_fe = pd.concat(fea_price, axis=1)
df_price_fe = adjust_price(df_price_fe) # add adjusted close value

In [215]:
df_fn = df_financial.copy()
acc_cols = ['NetSales', 'Profit', 'OperatingProfit', 'OrdinaryProfit']
df_fn[acc_cols] = df_fn[acc_cols].replace('－', 0).astype('float32')
# drop irregular financial quarter
df_fn = df_fn[~df_fn.TypeOfCurrentPeriod.isin(['4Q', '5Q', np.nan])].reset_index(drop=True)
df_fn['quarter'] = df_fn.TypeOfCurrentPeriod.map({"1Q": 1, "2Q": 2, "3Q": 3, "FY":4})
df_fn['FiscalYear'] = df_fn.CurrentFiscalYearEndDate.str.split('-', expand=True)[0].astype('int16')

fp_ymd = df_fn.CurrentPeriodEndDate.str.split('-', expand=True).astype('float32')
df_fn['fp_month'] = (fp_ymd[0] - 2016) * 12 + fp_ymd[1]
df_quarter = (
    df_fn[~(df_fn.CurrentPeriodEndDate.isna())]
        .groupby(['SecuritiesCode', 'CurrentPeriodEndDate'])
        .nth(-1)
)

df_quarter['quarter_span'] = df_quarter.groupby('SecuritiesCode').fp_month.diff().fillna(3)
df_fn = df_fn.join(df_quarter.quarter_span, on=['SecuritiesCode', 'CurrentPeriodEndDate'])
year_span = (df_quarter.groupby(['SecuritiesCode', 'FiscalYear']).quarter_span.sum()).rename('year_span')
df_fn = df_fn.join(year_span, on=['SecuritiesCode', 'FiscalYear'])
df_diff = df_quarter.groupby(['SecuritiesCode', 'FiscalYear'])[acc_cols].diff().fillna(
    df_quarter[acc_cols] / df_quarter.quarter.values[:, None]
)
df_diff /= (df_quarter["quarter_span"].values[:, None] / 3)

In [216]:
df_ma = df_diff[acc_cols].rolling(4, min_periods=1).mean()
df_diff = df_diff.add_prefix('diff_')
df_ma = df_ma.add_prefix('ma_')

df_fn = df_fn.merge(df_diff, on=['SecuritiesCode', 'CurrentPeriodEndDate'], how='left')
df_fn = df_fn.merge(df_ma, on=['SecuritiesCode', 'CurrentPeriodEndDate'], how='left')


In [217]:
df_fn[['TotalAssets', 'Equity']] = df_fn[['TotalAssets', 'Equity']].replace('－', 0).astype('float32')

In [218]:
amount_cols = ['TotalAssets', 'Equity']

amount_cols += ["diff_" + col for col in acc_cols]
amount_cols += ["ma_" + col for col in acc_cols]
amount_cols += acc_cols

key_cols = ["SecuritiesCode", "FiscalYear", "TypeOfCurrentPeriod"]
df_last_year = df_fn.groupby(key_cols)[amount_cols].nth(-1).reset_index()
df_last_year.FiscalYear += 1
df_last_fn = df_fn[key_cols].merge(df_last_year, on=key_cols, how='left')
df_last_ratio = (df_fn[amount_cols] - df_last_fn[amount_cols]) / df_last_fn[amount_cols]
df_last_ratio = df_last_ratio.add_prefix('feat_ratio_')


In [223]:
df_fn = pd.concat([df_fn, df_last_ratio], axis=1)

In [226]:
df_price_fe

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,...,feat_pct60_Close,feat_vl60_Close,feat_rmr60_Close,feat_wd60,feat_pct120_Close,feat_vl120_Close,feat_rmr120_Close,feat_wd120,CumulativeAdjustmentFactor,AdjustedClose
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,...,NaN,NaN,1.000000,1.000977,NaN,NaN,1.000000,1.000977,1.0,2742.0
1,20170105_1301,2017-01-05,1301,2743.0,2747.0,2735.0,2738.0,17900,1.0,NaN,...,NaN,NaN,0.999512,1.000977,NaN,NaN,0.999512,1.000977,1.0,2738.0
2,20170106_1301,2017-01-06,1301,2734.0,2744.0,2720.0,2740.0,19900,1.0,NaN,...,NaN,378.0000,1.000000,1.000977,NaN,378.0000,1.000000,1.000977,1.0,2740.0
3,20170110_1301,2017-01-10,1301,2745.0,2754.0,2735.0,2748.0,24200,1.0,NaN,...,NaN,427.0000,1.001953,1.000977,NaN,427.0000,1.001953,1.000977,1.0,2748.0
4,20170111_1301,2017-01-11,1301,2748.0,2752.0,2737.0,2745.0,9300,1.0,NaN,...,NaN,501.2500,1.000977,1.000977,NaN,501.2500,1.000977,1.000977,1.0,2745.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,20190327_9997,2019-03-27,9997,892.0,897.0,883.0,887.0,180000,1.0,NaN,...,-0.101318,74.8750,0.949707,1.004883,-0.317627,79.7500,0.842285,1.004883,1.0,887.0
1048571,20190328_9997,2019-03-28,9997,876.0,877.0,850.0,853.0,265500,1.0,NaN,...,-0.071838,74.7500,0.914062,1.003906,-0.349365,78.7500,0.812500,1.004883,1.0,853.0
1048572,20190329_9997,2019-03-29,9997,857.0,872.0,853.0,858.0,289400,1.0,NaN,...,-0.075439,74.9375,0.920898,1.003906,-0.360596,78.3125,0.820801,1.004883,1.0,858.0
1048573,20190401_9997,2019-04-01,9997,867.0,883.0,855.0,863.0,408800,1.0,NaN,...,-0.143799,75.3750,0.928223,1.003906,-0.345215,77.8750,0.828613,1.004883,1.0,863.0


In [231]:
feature_stocklist_cols = ['Section/Products', '33SectorCode', 
    '17SectorCode', 'NewIndexSeriesSizeCode']

,SecuritiesCode,EffectiveDate,Name,Section/Products,NewMarketSegment,33SectorCode,33SectorName,17SectorCode,17SectorName,NewIndexSeriesSizeCode,NewIndexSeriesSize,TradeDate,Close,IssuedShares,MarketCapitalization,Universe0,feat_cat_Section/Products,feat_cat_33SectorCode,feat_cat_17SectorCode,feat_cat_NewIndexSeriesSizeCode
0,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,20211230.0,3080.0,1.092828e+07,3.365911e+10,True,First Section (Domestic),50,1,7
1,1305,20211230,Daiwa ETF-TOPIX,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,2097.0,3.634636e+09,7.621831e+12,False,ETFs/ ETNs,-,-,-
2,1306,20211230,NEXT FUNDS TOPIX Exchange Traded Fund,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,2073.5,7.917718e+09,1.641739e+13,False,ETFs/ ETNs,-,-,-
3,1308,20211230,Nikko Exchange Traded Index Fund TOPIX,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,2053.0,3.736943e+09,7.671945e+12,False,ETFs/ ETNs,-,-,-
4,1309,20211230,NEXT FUNDS ChinaAMC SSE50 Index Exchange Trade...,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,44280.0,7.263200e+04,3.216145e+09,False,ETFs/ ETNs,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4412,9994,20211230,YAMAYA CORPORATION,First Section (Domestic),Standard Market,6100,Retail Trade,14,RETAIL TRADE,7,TOPIX Small 2,20211230.0,2447.0,1.084787e+07,2.654474e+10,True,First Section (Domestic),6100,14,7
4413,9995,20211230,"GLOSEL Co.,Ltd.",First Section (Domestic),Prime Market,6050,Wholesale Trade,13,COMMERCIAL & WHOLESALE TRADE,7,TOPIX Small 2,20211230.0,410.0,2.642680e+07,1.083499e+10,False,First Section (Domestic),6050,13,7
4414,9996,20211230,"Satoh&Co.,Ltd.",JASDAQ(Standard / Domestic),Standard Market,6050,Wholesale Trade,13,COMMERCIAL & WHOLESALE TRADE,-,-,20211230.0,1488.0,9.152640e+06,1.361913e+10,False,JASDAQ(Standard / Domestic),6050,13,-
4415,9997,20211230,"BELLUNA CO.,LTD.",First Section (Domestic),Prime Market,6100,Retail Trade,14,RETAIL TRADE,6,TOPIX Small 1,20211230.0,709.0,9.724447e+07,6.894633e+10,True,First Section (Domestic),6100,14,6
